# <center>企业级大模型部署推理管理工具</center>

## <center>Part 1. Vllm 框架基础入门与本地私有化部署</center>

&emsp;&emsp;对开源大模型来说，虽然模型权重开源，但并不意味着这样的模型就可以开箱即用，而是需要一些框架来支撑其运行和推理。无论哪个公司旗下的模型，现在都在兼容同一个接入规范，即`OpenAI`兼容接口，因为这种`API`规范几乎可以与任何`SDK`或者客户端无缝集成，所以对大模型部署框架而言，是否兼容`OpenAI`规范也就成为了衡量一个其是否成熟、是否可以被广泛使用的重要指标。所以大家都关注的问题就是：部署与`OpenAI API`规范兼容模型的最佳框架是什么？

&emsp;&emsp;如下所示，目前主流的大模型部署框架以`llama.cpp`、`Ollama`和`Vllm`为主，其各自项目在`Github`上的`Stars`增长曲线如下所示。其中`Ollama`的`Stars`增长曲线是断档式领先，而`llama.cpp`和`Vllm`的`Stars`增长曲线则相对平缓，处于一个稳步增长的趋势。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181034368.png" width=60%></div>

&emsp;&emsp;之所以会呈现出这种现状，其实与各个框架的定位有本质的区别。首先，`llama.cpp` 是使用没有任何依赖关系的纯 C/C++ 实现，性能很高，可定制化和优化项非常多，但也因为底层是`C`语言，直接导致上手难度极高。`Ollama` 之所以能够受到最多开发者的关注，一个最根本的原因就是其部署和使用太简单了，兼容多种操作系统，且都提供了一键安装、单命令启动的快捷方式，可以极大降低初学者使用开源大模型的门槛，同时`Ollama`框架提供原生 `REST API` 和 `OpenAI API` 兼容性，也可以非常轻松的接入到其他的客户端中。

# 1. Vllm 框架整体概览

&emsp;&emsp;如果说`Ollama`的优势在于其简洁性，那么`Vllm`则是一个另辟蹊径、优先考虑性能和可扩展性的大模型部署框架，其核心的优化点在`高效内存管理`、`持续批处理功能`和`张量并行性`，从而在生产环境中的高吞吐量场景中表现极佳，同时这也是为什么`Vllm`框架是目前最适用于企业真实生产环境部署的根本原因。

&emsp;&emsp;`Vllm` 底层是基于`Pytorch` 构建，其`Gtihub` 开源地址为：https://github.com/vllm-project/vllm 

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181137533.png" width=60%></div>

&emsp;&emsp;从各种基准测试数据来看，同等配置下，使用 `vLLM` 框架与 `Transformer` 等传统推理库相比，其吞吐量可以提高一个数量级，这归功于以下几个特性：

- **高级 GPU 优化**：利用 `CUDA` 和 `PyTorch` 最大限度地提高 `GPU` 利用率，从而实现更快的推理速度。`Ollama`其实是对`CPU-GPU`的混合应用，但`vllm`是针对纯`GPU`的优化。
- **高级内存管理**：通过`PagedAttention`算法实现对 `KV cache`的高效管理，减少内存浪费，从而优化大模型的运行效率。
- **批处理功能**：支持连续批处理和异步处理，从而提高多个并发请求的吞吐量。
- **安全特性**：内置 `API` 密钥支持和适当的请求验证，不像其他完全跳过身份验证的框架。
- **易用性**：`vLLM` 与 `HuggingFace` 模型无缝集成，支持多种流行的大型语言模型，并兼容 `OpenAI` 的 `API` 服务器。

&emsp;&emsp;以上核心的优化点原理以及应用方法，我们将在接下来的课程进行详细的讲解和实践。本节课程我们需要先进行`Vllm` 框架的私有化部署的学习。

&emsp;&emsp; `vllm` 框架主要支持纯文本和多模态两种模式的模型，其中对支持的纯文本类语言模型可以在这里看到：https://docs.vllm.ai/en/latest/models/supported_models.html#supported-text-models

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511604.png" width=60%></div>

&emsp;&emsp;其次是多模态模型，`Vllm`框架目前已经支持文本、图片、视频和音频四种模态的输入输出格式兼容规范，但是多模态模型官方并没有给出具体的模型列表，所以这里我们要进入到源码中对模型的集成文件中去匹配：https://github.com/vllm-project/vllm/tree/main/vllm/model_executor/models

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181304597.png" width=60%></div>

&emsp;&emsp;根据各个模型支持`.py`文件中的代码细节，这里梳理了一份目前最新版本`Vllm 0.8.4`支持的多模态模型如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>多模态模型支持列表</font></p>
<div class="center">

| .py 文件                          | 模型名称                          | 描述                                       |
|----------------------------------|----------------------------------|------------------------------------------|
| aya_vision.py                    | aya-vision                         | https://huggingface.co/CohereLabs/aya-vision-8b       |
| blip.py / blip2.py              | BLIP                            | 经典视觉-语言模型（BLIP 系列）。               |
| clip.py                          | CLIP                            | 经典的多模态对比学习模型（CLIP）。             |
| deepseek_vl2.py                  | DeepSeek                       | DeepSeek 的多模态版本（VL 表示 Vision-Language）。 |
| florence2.py                    | 微软模型                        | 微软的多模态模型。https://huggingface.co/microsoft/Florence-2-large                           |
| fuyu.py                          | Fuyu                           | Adept 的多模态模型（Fuyu 系列）。              |
| gemma3_mm.py                     | Gemma                          | mm 表示 Multimodal（Gemma 的多模态扩展）。    |
| glm4v.py                         | GLM-4                          | GLM-4 的多模态版本（v 表示 Vision）。         |
| h2ovl.py                         | H2O                            | H2O 的视觉-语言模型（vl 后缀）。              |
| idefics2_vision_model.py / idefics3.py | Idefics                     | Meta 的多模态模型（Idefics 系列）。       |
| internvl.py                      | InternVL-Chat                   | 上海 AI Lab 的多模态模型。                   |
| kimi_vl.py                       | Kimi-VL-A3B                        | 新增的 Kimi-VL 模型  |
| llava.py / llava_next.py / llava_next_video.py / llava_onevision.py | LLaVA                        | LLaVA 系列及其变种（开源视觉-语言模型）。 |
| mllama4.py                       | LLaMA-4                       | 多模态版本的 LLaMA-4。                      |
| molmo.py                         | Molmo                     | 提交记录涉及多模态数据处理。                   |
| moonvit.py                       | Kimi-VL                        | 与 Kimi-VL 同时新增的多模态模型。             |
| nvlm_d.py                        | NVLM-D-72B                         | NVIDIA 的多模态模型（vl 相关）。              |
| paligemma.py                     | PaLI-Gemma                     | Google 的 PaLI-Gemma 多模态模型。            |
| phi3v.py / phi4mm.py / phi4mm_audio.py | Phi-3 / Phi-4               | Phi-3 和 Phi-4 的多模态版本（支持视觉或音频）。 |
| prithvi_geospatial_mae.py        | Prithvi-EO-1.0                    | 地理空间多模态模型（NASA 合作项目）。         |
| qwen2_5_vl.py / qwen2_audio.py / qwen2_vl.py / qwen_vl.py | 通义千问                     | 通义千问的多模态版本（视觉或音频）。      |
| siglip.py                        | SigLIP                        | Google 的 SigLIP（多模态对比学习）。         |
| skyworkr1v.py                    | Skywork-R1V-38B                       | 幻方多模态模型（v 后缀）。                   |
| smolvlm.py                       | SmolVLM                     | 轻量级多模态模型（新增支持）。     |
| whisper.py                       | whisper-large-v3                   | OpenAI 的语音-文本多模态模型（虽然主要处理音频，但属于跨模态）。 |
</div>

# 2. Linux 操作系统部署 Vllm

&emsp;&emsp; 首先需要重点说明的是：<font color="red">`Vllm` 框架仅支持`Linux` 操作系统，官方并没有提供`Windows`的兼容版本</font>。同时除了有操作的限制，<font color="red">对运行的`Python`版本也要求在`Python 3.8` ~ `Python 3.12` 之间</font>。这两个条件限制为部署`Vllm`的先决条件，即必须满足才可以顺利使用`Vllm`启动大模型并提供推理服务。

&emsp;&emsp;这个策略非常符合企业级的部署策略，`Vllm` 框架作为目前在实际生产环境中使用最为广泛的框架之一，其对`Linux` 操作系统的支持，以及对`Python` 版本的限制，都是为了确保在生产环境中能够稳定运行，并且能够提供高性能的推理服务。（在生产环境中，没有任何一家企业会在`Windows` 操作系统上部署模型服务或者应用服务）

&emsp;&emsp;因此，我们接下来就严格按照操作系统和`Python` 版本的要求，来进行`Vllm` 框架部署和使用的详细讲解和实践。


&emsp;&emsp; 这里我们采用的`Linux` 操作系统是`Ubuntu 22.04`，同时配置四张`3090` 显卡来进行`Vllm` 框架的部署和使用。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511611.png" width=80%></div>

&emsp;&emsp; `Vllm` 工具已经通过编译并上传到`PYPI` 仓库以及主流的包管理平台中，所以可以直接使用如`pip`、`conda` 等工具进行快速安装。这里我们使用比较常用的`conda` 包版本管理工具进行安装部署。


- **Step 1. 安装`conda` 包版本管理工具**

&emsp;&emsp;首先需要检查当前使用的操作系统是否已经安装了`conda` 包版本管理工具，可以通过`conda --version` 命令查看`Conda` 版本，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171523767.png" width=80%></div>

&emsp;&emsp;如果出现`Conda not found` 等报错，需要先安装`Conda` 环境，再执行接下来的步骤。


- **Step 2. 创建Python虚拟环境**

&emsp;&emsp;`vllm`官方要求的是`Python 3.9` ~ `Python 3.12` 之间的版本，这里我们选择`Python 3.12` 版本。执行如下命令进行创建：

```bash
    conda create --name vllm python=3.12
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511612.png" width=80%></div>

- **Step 3. 激活虚拟环境**


&emsp;&emsp;创建完虚拟环境后，使用`Conda` 激活虚拟环境，通过`conda activate vllm` 命令激活，如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511613.png" width=80%></div>

&emsp;&emsp;接下来，使用`pip` 安装`vllm` 框架，执行如下命令：

```bash
    pip install vllm
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511614.png" width=80%></div>

&emsp;&emsp;此时耐心等待安装完成即可。待安装完成后，可以使用`pip show vllm` 命令查看`vllm` 框架的安装信息，可以明确看到当前安装的`vllm` 版本号。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511615.png" width=80%></div>

&emsp;&emsp;至此，`Vllm` 框架的安装就已经完成了。整个过程就是非常常规的依赖包安装过程，唯一需要注意的就是`Python` 版本的要求。

&emsp;&emsp;在安装完成`Vllm` 框架后，就可以开始进行大模型推理服务的启动和调用了。`Vllm` 框架提供了两种启动并调用大模型生成推理服务的方式，分别是<font color="red">离线推理</font>和<font color="red">在线推理</font>。其中：

- **离线推理**： 离线推理类似于使用 `Pytorch` 模块一样，当我们需要使用大模型生成推理服务时，先加载模型，然后使用输入数据运行该模型，并获取输出结果。
- **在线推理**： 在线推理类似于有一个服务器，可以先启动大模型，然后等待来自客户端的请求，一旦接收到请求，就会使用大模型生成推理服务，并返回结果，并且可以同时处理多个请求。

&emsp;&emsp;这是两种不同推理方式最本质的区别，毫无疑问<font color="red">在线推理是更加符合实际生产环境的</font>。但为了帮助大家更好的理解`Vllm` 框架，这里我们还是先从离线推理开始讲解，然后再重点讲解在线推理。


# 3. vllm 离线推理

&emsp;&emsp;无论是在线推理还是离线推理，都采用的是`API` 接口的方式来调用大模型生成推理服务。对离线推理来说，其加载模型、传输数据、获取结果的`API` 调用形式如下所示：

```python
    from vllm import LLM
    # 加载模型
    llm = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")
    
    # 传输数据
    outputs = llm.generate("你好，我是木羽")

    # 获取结果
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
```



&emsp;&emsp;代码看起来简单，但如果想要跑通这几行代码，则需要我们提前做两件非常关键的准备工作：<font color="red">其一是需要在`Linux` 服务器上下载大模型权重，其二则是需要配置一个可以加载服务器虚拟环境（即上一步创建的`vllm` 虚拟环境）的`Python IDE`解释器</font>。


&emsp;&emsp;我们就以`DeepSeek-R1-Distill-Qwen-32B` 大模型为例，默认情况下，`LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")` 这行命令的执行逻辑是：先检测当前服务器中是否存在`deepseek-ai/DeepSeek-R1-Distill-Qwen-32B` 的模型权重，如果存在则直接加载该目录下的模型权重，否则会从`Hugging Face` 上下载大模型权重。其存在的主要问题是：国内的服务器是没有办法访问`Hugging Face` 网站的，所以一定会报错。因此，国内的开发者需要采用更有效的方式来确保大模型权重的正确下载，即使用国内镜像源`ModelScope` 来执行模型权重的本地化存储。

&emsp;&emsp;回到`Linux` 服务器中，首先通过如下命令安装`ModelScope` 的`pip` 包：

```bash
    pip install modelscope
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511620.png" width=80%></div>

&emsp;&emsp;接下来，进入到`ModelScope` 的主页中：https://www.modelscope.cn/home

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613863.png" width=80%></div>


&emsp;&emsp;通过关键词找到对应的模型详情页，比如我这里以`DeepSeek-R1-Distill-Qwen-32B` 为例，点击进入详情页，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613865.png" width=80%></div>

&emsp;&emsp;进入到模型详情页后，点击`模型文件`，然后再找到`下载模型`按钮，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613866.png" width=80%></div>

&emsp;&emsp;在弹出来的会话框中，找到`SDK 下载` 标签，复制对应的模型服务指针标识：如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613867.png" width=80%></div>

&emsp;&emsp;接下来回到`Linux` 服务器中，通过`vim` 新建一个`model_download.py` 文件：

```python
    vim model_download.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511621.png" width=80%></div>

&emsp;&emsp;然后，按`i` 键进入编辑模式，将复制的代码粘贴到文件中：

```python
    from modelscope import snapshot_download

    # model_dir = snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-32B', cache_dir='/root/autodl-tmp', revision='master')
    model_dir = snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-32B', cache_dir='/home/08_vllm', revision='master')
```

&emsp;&emsp;注意，这里`cache_dir` 参数指定的路径是模型权重存储的路径，这里我们指定为`/home/08_vllm` 目录，即模型权重会存储到该目录下。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511622.png" width=80%></div>

&emsp;&emsp;修改完成后，按`Esc` 键退出编辑模式，再按`Shift+:` 键进入命令模式，输入`wq` 保存并退出。执行如下命令，运行`model_download.py` 文件：

```bash
    python model_download.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511623.png" width=80%></div>

&emsp;&emsp;耐心等待模型权重下载完成即可。待下载完成后，可以看到`/home/08_vllm` 目录下会多出一个`deepseek-ai` 文件夹，里面存放的就是`DeepSeek-R1-Distill-Qwen-32B` 大模型的权重文件。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171622553.png" width=80%></div>

&emsp;&emsp;按照上述流程中`model_download.py`下载模型的方式，可以允许我们随意下载任意在`ModelScope`中托管的模型权重至本地，因此这里依次下载了`Qwen2.5`系列和`DeepSeek R1` 系列的不同尺寸的模型，大家也可以根据自己的需求灵活选择模型。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181342835.png" width=80%></div>

&emsp;&emsp;至此，大模型权重就已经下载完成了。接下来，我们需要做的是配置一个可以加载服务器虚拟环境（即上一步创建的`vllm` 虚拟环境）的`Python IDF`解释器。这一步就会根据大家实际的开发情况产生不同的配置方法，总的来说会分成以下两种情况：

1. 部署大模型的服务器和你要执行代码调用的`Python IDE`是同一台服务器，则可以直接选择到对应的虚拟环境并执行代码；
2. 部署大模型的服务器和你要执行代码调用的`Python IDE`不是同一台服务器，则需要通过`SSH` 连接到部署大模型的服务器，然后选择到对应的虚拟环境并执行代码。

&emsp;&emsp;同一台服务器的情况非常简单，直接打开`Python IDE`，选择到对应的虚拟环境并执行代码即可。这里重点介绍第二种情况，即部署大模型的服务器和你要执行代码调用的`Python IDE`不是同一台服务器。同时这也是最常见的企业开发环境。


&emsp;&emsp;这类情况用最通俗的理解是：我们把大模型部署在了局域网/租赁的云服务器中，希望可以在本地电脑上执行代码调用模型服务。这类情况我们要做如下配置：（以`Jupyter Notebook`为例）:

&emsp;&emsp;首先进入在局域网/租赁的云服务器的`Linux` 服务器中，在`vllm` 虚拟环境中安装`ipykernel` 和`jupyter` 包，执行如下命令：
```bash
    pip install ipykernel jupyter
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511616.png" width=80%></div>


&emsp;&emsp;安全起见，对连接时的密码进行加密处理，否则明文写在配置文件中，容易造成数据安全风险，依次执行如下操作：
```bash
    from jupyter_server.auth import passwd
    passwd()
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753935.png" width=80%></div>

&emsp;&emsp;完成密码加密后，执行如下命令生成Jupyter Lab 的配置文件（jupyter_lab_config.py）：
```bash
    jupyter lab --generate-config
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753934.png" width=80%></div>

&emsp;&emsp;使用`Vim`编辑器，找到如下配置，执行修改：
```bash
    c.ServerApp.allow_origin = '*'
    c.ServerApp.allow_remote_access = True
    c.ServerApp.ip = '0.0.0.0'
    c.ServerApp.open_browser = False  
    c.ServerApp.password = '加密后的密码'（上一步复制的加密串）
    c.ServerApp.port = 8002 
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753936.png" width=80%></div>

&emsp;&emsp;全部配置完成后，在服务器端启动Jupyter Lab服务，通过如下命令后台启动：
```bash
    nohup jupyter lab --allow-root > jupyterlab.log 2>&1 &
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753937.png" width=80%></div>

&emsp;&emsp;然后，将虚拟环境的`Kernel`写入`Jupyter Lab`，执行如下命令，创建一个`ipykernel` 内核：
```bash
    python -m ipykernel install --user --name vllm --display-name "vllm"
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511617.png" width=80%></div>

&emsp;&emsp;接下来，在浏览器中打开`Jupyter Notebook` 服务，在`Jupyter kernel` 中就会出现选择`vllm` 内核的选项，如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511618.png" width=80%></div>

&emsp;&emsp;选择`vllm` 内核并创建一个新的`notebook` 笔记本，可以通过`! pip show vllm` 快速验证是否正常加载到了`vllm` 的虚拟环境：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511619.png" width=80%></div>

In [1]:
# ! pip show vllm

Name: vllm
Version: 0.8.4
Summary: A high-throughput and memory-efficient inference and serving engine for LLMs
Home-page: https://github.com/vllm-project/vllm
Author: vLLM Team
Author-email: 
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or mana

&emsp;&emsp;如果正常加载到了`vllm` 的虚拟环境，则可以看到`vllm` 的版本号，这里可以看到，与我们在服务器上安装的版本号是一致的。说明我们已经在本地电脑上成功加载到了`vllm` 的虚拟环境。

&emsp;&emsp;接下来，我们可以通过离线推理的`API` 接口来调用大模型生成推理服务。我们会对对话模型和推理模型依次展开详细的测试。


&emsp;&emsp;首先以`Qwen2.5`系列为例测试在`vllm`框架下对对话类模型的使用。默认情况下，`Vllm` 从`huggingface`下载模型权重，并使用`transformers`库加载模型。这里我们的模型是从`ModelScope`下载的，所以在调用离线推理`API`时，需要搭配`trust_remote_code=True`参数，其次，对`model` 参数，指的是模型权重文件的本地路径，因此需要大家根据模型下载路径进行修改。即实例化代码如下图所示：

In [3]:
from vllm import LLM

llm = LLM(model="/root/autodl-tmp/vllm/Qwen/Qwen3-4B",
          trust_remote_code=True,
          max_model_len=4096,
)

INFO 10-05 16:59:33 [utils.py:233] non-default args: {'trust_remote_code': True, 'max_model_len': 4096, 'disable_log_stats': True, 'model': '/root/autodl-tmp/vllm/Qwen/Qwen3-4B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-05 16:59:33 [model.py:547] Resolved architecture: Qwen3ForCausalLM
INFO 10-05 16:59:33 [model.py:1510] Using max model len 4096
INFO 10-05 16:59:33 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:33 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:33 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/root/autodl-tmp/vllm/Qwen/Qwen3-4B', speculative_config=None, tokenizer='/root/autodl-tmp/vllm/Qwen/Qwen3-4B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backen

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(EngineCore_DP0 pid=20463) INFO 10-05 16:59:39 [default_loader.py:267] Loading weights took 3.70 seconds
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:40 [gpu_model_runner.py:2653] Model loading took 7.5552 GiB and 3.953302 seconds
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:46 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/f2d0106e42/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:46 [backends.py:559] Dynamo bytecode transform time: 5.78 s
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:49 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.091 s
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:50 [monitor.py:34] torch.compile takes 5.78 s in total
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:52 [gpu_worker.py:298] Available KV cache memory: 12.19 GiB
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:52 [kv_cache_utils.py:1087] GPU KV cache size: 88,784 tokens
(EngineCore_DP0 pid=20463) INF

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:04<00:00, 16.08it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 19.32it/s]


(EngineCore_DP0 pid=20463) INFO 10-05 16:59:59 [gpu_model_runner.py:3480] Graph capturing finished in 7 secs, took 0.84 GiB
(EngineCore_DP0 pid=20463) INFO 10-05 16:59:59 [core.py:210] init engine (profile, create kv cache, warmup model) took 19.23 seconds
INFO 10-05 17:00:00 [llm.py:306] Supported_tasks: ['generate']


In [7]:
# 释放显存

# 删除 LLM 对象
del llm

# 触发垃圾回收
import gc
gc.collect()

# 清空 CUDA 缓存
import torch
torch.cuda.empty_cache()

&emsp;&emsp;`LLM` 类是运行离线推理的主要类。当使用`LLM`类进行模型初始化的加载时，有如下几个最初级且重要的参数需要大家掌握：

1. **device**: 该参数会指定模型加载的设备，vllm 支持在 `cuda, neuron, cpu, tpu, xpu, hpu`，默认是`auto`, 即初始化时会自动识别当前系统中可用的设备。
2. **tensor_paralleis***ze"：该参数会指定模型可以通过`GPU`的`Tensor`并行运行，默认是`1`，即不使用`GPU`的`Tensor`并行，但如果加载的模型参数超过单个`GPU`的显存，即使服务器上有多个`GPU`，也会报错`Out of memory`，并不会自动使用其他`GPU`进行并行。
3. **gpu_memory_utilization**：该参数会指定模型加载时，`GPU`的显存取值范围为`0~1`，使用率，默认是`0.90`，即`GPU`的显存使用率不会0过`95%`，但该参数会直接占用`GPU`的显存，以避免`GPU`的显存被其他进占用。


&emsp;&emsp;比如对`Qwen2.5-7B`模型，采用默认配置其显存占用情况如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181430516.png" width=80%></div>

&emsp;&emsp;如果模型较大，则可以指定多块`GPU`，比如如下所示的参数调整：

In [1]:
from vllm import LLM

llm = LLM(model="/root/autodl-tmp/vllm/Qwen/Qwen3-4B",
          trust_remote_code=True,
          # tensor_parallel_size=2,
          gpu_memory_utilization=0.8,
          max_model_len=1024,
)

INFO 10-05 17:25:06 [__init__.py:216] Automatically detected platform cuda.
INFO 10-05 17:25:09 [utils.py:233] non-default args: {'trust_remote_code': True, 'max_model_len': 1024, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'model': '/root/autodl-tmp/vllm/Qwen/Qwen3-4B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-05 17:25:09 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-05 17:25:09 [model.py:1510] Using max model len 1024
INFO 10-05 17:25:09 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:10 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:10 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/root/autodl-tmp/vllm/Qwen/Qwen3-4B', speculative_config=None, tokenizer='/root/autodl-tmp/vllm/Qwen/Qwen3-4B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_add

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(EngineCore_DP0 pid=31880) INFO 10-05 17:25:15 [default_loader.py:267] Loading weights took 2.01 seconds
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:16 [gpu_model_runner.py:2653] Model loading took 7.5552 GiB and 2.242479 seconds
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:22 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/7faaa76617/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:22 [backends.py:559] Dynamo bytecode transform time: 5.73 s
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:25 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.125 s
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:26 [monitor.py:34] torch.compile takes 5.73 s in total
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:28 [gpu_worker.py:298] Available KV cache memory: 9.84 GiB
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:28 [kv_cache_utils.py:1087] GPU KV cache size: 71,616 tokens
(EngineCore_DP0 pid=31880) INFO

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:04<00:00, 15.96it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 19.65it/s]


(EngineCore_DP0 pid=31880) INFO 10-05 17:25:35 [gpu_model_runner.py:3480] Graph capturing finished in 7 secs, took 0.84 GiB
(EngineCore_DP0 pid=31880) INFO 10-05 17:25:35 [core.py:210] init engine (profile, create kv cache, warmup model) took 19.24 seconds
INFO 10-05 17:25:36 [llm.py:306] Supported_tasks: ['generate']


&emsp;&emsp;此时服务器的`GPU`资源占用情况就如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181435870.png" width=80%></div>

&emsp;&emsp;大家可以根据自己实际加载模型的大小及硬件资源情况进行灵活的调整，

&emsp;&emsp;在加载好模型实例后，便可以进行远程模型服务的调用。发送数据并获取模型推理响应的接口是`.generate`方法，如下代码所示：

In [2]:
outputs = llm.generate("你好，请你介绍一下你自己")

outputs

WARNING 10-05 17:26:13 [model.py:1389] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=0, prompt='你好，请你介绍一下你自己', prompt_token_ids=[108386, 37945, 56568, 109432, 107828], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='，你是谁，来自哪里，有什么能力？\n\n您好！我是通义千', token_ids=[3837, 105043, 100165, 3837, 101919, 101314, 3837, 104139, 99788, 26850, 111308, 6313, 104198, 31935, 64559, 99320], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]

&emsp;&emsp;响应结果返回的是一个`RequestOutput`对象，通过`RequestOutput.outputs[0].text`可以获取到模型的推理结果。

In [3]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '，你是谁，来自哪里，有什么能力？\n\n您好！我是通义千'


&emsp;&emsp;除了传入字符串，当输入一个列表时，`.generate`方法会执行批量推理，针对每一个`prompt`执行一次推理，并返回一个`RequestOutput`对象的列表。

In [4]:
text = [
    "你好，请你介绍一下你自己",
    "请问什么是机器学习",
    "请问如何理解大模型？"
]

In [5]:
outputs = llm.generate(text)

outputs

Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=1, prompt='你好，请你介绍一下你自己', prompt_token_ids=[108386, 37945, 56568, 109432, 107828], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='。 你好，我叫通义千问，是阿里巴巴集团旗下的通', token_ids=[1773, 220, 108386, 3837, 35946, 99882, 31935, 64559, 99320, 56007, 3837, 20412, 107076, 100338, 111477, 31935], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=0, multi_modal_placeholders={}),
 RequestOutput(request_id=2, prompt='请问什么是机器学习', prompt_token_ids=[109194, 106582, 102182, 100134], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='，它有哪些常见的类型，以及它们在实际应用中有什么用途？\n\n机器', token_ids=[3837, 99652, 104719, 102716, 31905, 3837, 101034, 104017, 18493, 99912, 99892, 15946, 104139, 105795, 26850, 102182], cumulative_logprob=None, logprobs=None, finish_reason=length, stop

In [6]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: '你好，请你介绍一下你自己', Generated text: '。 你好，我叫通义千问，是阿里巴巴集团旗下的通'
Prompt: '请问什么是机器学习', Generated text: '，它有哪些常见的类型，以及它们在实际应用中有什么用途？\n\n机器'
Prompt: '请问如何理解大模型？', Generated text: '大模型和小模型有什么区别？大模型的发展前景如何？\n\n理解大'


&emsp;&emsp;以上就是在不修改任何模型加载和推理代码的情况下，快速使用`vllm` 框架调用本地模型进行推理的方式。整个过程看并不是特别复杂。

&emsp;&emsp;接下来我们再看推理类模型的在`Vllm` 离线推理`API`中的使用，这里我们以`DeepSeek-R1-Distill-Qwen-7B`为例。

In [1]:
from vllm import LLM

llm = LLM(model="/root/autodl-tmp/vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B",
          trust_remote_code=True,
          # tensor_parallel_size=2,
          gpu_memory_utilization=0.8,
          max_model_len=4096,
)

INFO 10-05 18:19:39 [__init__.py:216] Automatically detected platform cuda.
INFO 10-05 18:19:42 [utils.py:233] non-default args: {'trust_remote_code': True, 'max_model_len': 4096, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'model': '/root/autodl-tmp/vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-05 18:19:42 [model.py:547] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-05 18:19:42 [model.py:1510] Using max model len 4096
INFO 10-05 18:19:42 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:43 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:43 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/root/autodl-tmp/vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B', speculative_config=None, tokenizer='/root/autodl-tmp/vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disab

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=48760) INFO 10-05 18:19:47 [default_loader.py:267] Loading weights took 0.86 seconds
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:47 [gpu_model_runner.py:2653] Model loading took 3.3461 GiB and 1.045258 seconds
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:52 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/cc8c9c85b5/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:52 [backends.py:559] Dynamo bytecode transform time: 4.02 s
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:54 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.324 s
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:54 [monitor.py:34] torch.compile takes 4.02 s in total
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:56 [gpu_worker.py:298] Available KV cache memory: 14.06 GiB
(EngineCore_DP0 pid=48760) INFO 10-05 18:19:56 [kv_cache_utils.py:1087] GPU KV cache size: 526,688 tokens
(EngineCore_DP0 pid=48760) IN

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:02<00:00, 28.87it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 31.13it/s]


(EngineCore_DP0 pid=48760) INFO 10-05 18:20:00 [gpu_model_runner.py:3480] Graph capturing finished in 4 secs, took 0.62 GiB
(EngineCore_DP0 pid=48760) INFO 10-05 18:20:00 [core.py:210] init engine (profile, create kv cache, warmup model) took 12.65 seconds
INFO 10-05 18:20:01 [llm.py:306] Supported_tasks: ['generate']


&emsp;&emsp;其显存占用情况与`Qwen2.5:7b` 并无差异：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181456307.png" width=80%></div>

&emsp;&emsp;接下来我们进行推理模型的调用测试。

In [2]:
outputs = llm.generate("你好，请你介绍一下你自己")

outputs

WARNING 10-05 18:20:07 [model.py:1389] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=0, prompt='你好，请你介绍一下你自己', prompt_token_ids=[151646, 108386, 37945, 56568, 109432, 107828], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='，然后介绍你的研究领域和成就。要突出你的学术背景和研究', token_ids=[3837, 101889, 100157, 103929, 99556, 100650, 33108, 102174, 1773, 30534, 102017, 103929, 104380, 102193, 33108, 99556], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]

In [3]:
outputs = llm.generate("请问什么是黑洞？")

outputs

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=1, prompt='请问什么是黑洞？', prompt_token_ids=[151646, 109194, 106582, 118718, 11319], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='并解释为什么“黑洞”这个名称会被误用。\n\n首先，我需要', token_ids=[62926, 104136, 100678, 2073, 118718, 854, 99487, 29991, 106764, 29056, 11622, 3407, 101140, 3837, 35946, 85106], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]

In [4]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '并解释为什么“黑洞”这个名称会被误用。\n\n首先，我需要'


&emsp;&emsp;这里能看到从返回结果上看，当改用了推理模型后，其返回的`text`字段中的内容会被截断的非常短，这是因为推理模型会包含思考过程，所以需要对`generate`接口要求输出的`Token`数需求更大，因此这里要引入一个`SamplingParams` 的概念。默认情况下，`vLLM` 的离线推理`API`会使用模型原生定义的采样参数，也就是本地存储权重中的`generation_config.json`文件中的配置。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181521542.png" width=80%></div>

&emsp;&emsp;当然，如果某些开源的模型权重中没有提供该文件，`Vllm`会根据其定义的`SamplingParams`类自动指定默认值。其源码位置如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181524308.png" width=80%></div>

&emsp;&emsp;其中包含的全部可定义参数如下表所示:

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>控制模型输出采样参数列表</font></p>
<div class="center">

| 参数                          | 描述                                                                                     |
|-----------------------------|----------------------------------------------------------------------------------------|
| `n`                         | 要返回的输出序列数量。                                                                  |
| `best_of`                  | 从提示生成的输出序列数量。从这些 `best_of` 序列中返回前 `n` 个序列。`best_of` 必须大于或等于 `n`。默认情况下，`best_of` 设置为 `n`。警告：此功能仅在 V0 中支持。 |
| `presence_penalty`         | 浮点数，根据新 token 是否出现在已生成的文本中对其进行惩罚。值 > 0 鼓励模型使用新 token，值 < 0 鼓励模型重复 token。 |
| `frequency_penalty`        | 浮点数，根据新 token 在已生成文本中的频率对其进行惩罚。值 > 0 鼓励模型使用新 token，值 < 0 鼓励模型重复 token。 |
| `repetition_penalty`       | 浮点数，根据新 token 是否出现在提示和已生成文本中对其进行惩罚。值 > 1 鼓励模型使用新 token，值 < 1 鼓励模型重复 token。 |
| `temperature`              | 浮点数，控制采样的随机性。较低的值使模型更确定，较高的值使模型更随机。零表示贪婪采样。 |
| `top_p`                    | 浮点数，控制考虑的 top token 的累积概率。必须在 (0, 1] 之间。设置为 1 以考虑所有 token。 |
| `top_k`                    | 整数，控制考虑的 top token 数量。设置为 -1 以考虑所有 token。                       |
| `min_p`                    | 浮点数，表示相对于最可能 token 的概率，考虑 token 的最小概率。必须在 [0, 1] 之间。设置为 0 以禁用此功能。 |
| `seed`                     | 用于生成的随机种子。                                                                    |
| `stop`                     | 停止生成的字符串列表。当生成这些字符串时，生成将停止。返回的输出将不包含停止字符串。 |
| `stop_token_ids`          | 停止生成的 token 列表。当生成这些 token 时，生成将停止。返回的输出将包含停止 token，除非停止 token 是特殊 token。 |
| `bad_words`                | 不允许生成的单词列表。更准确地说，只有当下一个生成的 token 可以完成序列时，才不允许对应 token 序列的最后一个 token。 |
| `include_stop_str_in_output` | 是否在输出文本中包含停止字符串。默认值为 False。                                      |
| `ignore_eos`               | 是否忽略 EOS token，并在生成 EOS token 后继续生成 token。                           |
| `max_tokens`               | 每个输出序列生成的最大 token 数量。                                                    |
| `min_tokens`               | 每个输出序列生成的最小 token 数量，直到可以生成 EOS 或 stop_token_ids。               |
| `logprobs`                 | 每个输出 token 返回的 log 概率数量。当设置为 None 时，不返回概率。如果设置为非 None 值，结果将包括指定数量的最可能 token 的 log 概率，以及选择的 token。注意，实施遵循 OpenAI API：API 将始终返回采样 token 的 log 概率，因此响应中可能有多达 logprobs+1 个元素。 |
| `prompt_logprobs`          | 每个提示 token 返回的 log 概率数量。                                                  |
| `detokenize`               | 是否对输出进行反分词。默认值为 True。                                                  |
| `skip_special_tokens`      | 是否在输出中跳过特殊 token。                                                            |
| `spaces_between_special_tokens` | 是否在输出中的特殊 token 之间添加空格。默认值为 True。                               |
| `logits_processors`        | 修改 logits 的函数列表，基于先前生成的 token，并可选地将提示 token 作为第一个参数。   |
| `truncate_prompt_tokens`    | 如果设置为整数 k，将仅使用提示的最后 k 个 token（即左截断）。默认值为 None（即不截断）。 |
| `guided_decoding`          | 如果提供，引擎将根据这些参数构建引导解码 logits 处理器。默认值为 None。               |
| `logit_bias`               | 如果提供，引擎将构建一个应用这些 logit 偏置的 logits 处理器。默认值为 None。         |
| `allowed_token_ids`        | 如果提供，引擎将构建一个 logits 处理器，仅保留给定 token ids 的分数。默认值为 None。 |
| `extra_args`               | 任意额外参数，可供自定义采样实现使用。未被任何树内采样实现使用。                       |

&emsp;&emsp;所以与`Qwen2.5:7b`不同的是，`DeepSeek-R1-Distill-Qwen-7B` 没有设置`max_tokens`，从而导致会用`Vllm`的默认值。因此，对大模型推理生成定制化的配置方法，则是按照上表中的参数说明定义`sampling_params`实例，设置更长的`max_tokens`参数，因为推理模型包含思考过程，需要输出更多的 `Token`，如下所示：

In [5]:
# vllm_model.py
from vllm import SamplingParams

# 根据 DeepSeek 官方的建议，temperature应在 0.5-0.7，推荐 0.6 
sampling_params = SamplingParams(max_tokens=8192, temperature=0.6, top_p=0.95)

&emsp;&emsp;构建好自定义的采样参数后，在`generate`方法中与输入的问题同步传递，代码如下：

In [14]:
outputs = llm.generate("你好，请你介绍一下你自己。", sampling_params=sampling_params)

outputs

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=8, prompt='你好，请你介绍一下你自己。', prompt_token_ids=[151646, 108386, 37945, 56568, 109432, 107828, 1773], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='这是一个比较基础的问题，不需要详细解答。\n\n好的，我现在要尝试解决这个问题。为了确保我能够正确回答，我会先回想一下自己相关的知识和经验。首先，我需要确定自己是哪个年龄段的人，这可能有助于了解我是否需要使用特定的表达方式。\n\n接下来，我需要了解自己在哪些领域有知识和经验，这有助于我准确地表达我的背景和兴趣。例如，如果我之前在教育领域有经验，或者在某个特定的行业有工作，这些都会对我回答的问题有帮助。\n\n然后，我需要思考自己是否需要学习或掌握某些特定的表达方式。例如，如果我之前在某个地方有特定的口语表达习惯，或者在某个领域有特定的术语，这些可能会影响我的回答质量。\n\n最后，我需要确认自己是否需要进行语言上的调整，以确保回答既准确又自然。例如，如果我使用了一些比较正式或复杂的词汇，可能会影响回答的自然流畅性。\n\n现在，我来分析一下自己可能需要学习或掌握的方面。首先，语言表达能力：我是否需要提高自己的口语能力，或者是否需要学习一些特定的表达方式？\n\n其次，文化背景：我是否需要了解自己所处的国家或地区的文化背景，以便更好地表达自己？\n\n第三，专业领域：我是否需要了解自己所从事的行业或相关领域的知识，以便更好地回答相关问题？\n\n第四，职业发展：我是否需要了解自己的职业目标，以便更好地表达自己？\n\n第五，自我认知：我是否需要了解自己的个人背景和性格特点，以便更好地表达自己？\n\n现在，我来逐一分析这些方面是否需要学习或掌握。\n\n首先，语言表达能力：我是否需要提高自己的口语能力？这可能取决于我回答的问题是否需要表达得更自然流畅。如果我之前在某个地方有特定的口语表达习惯，可能会影响我的回答质量。\n\n其次，文化背景：我是否需要了解自己所处的国家或地区的文化背景？这可能取决于

In [7]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '请说明你的职业背景和经验，以及你的兴趣爱好。请说明你为什么选择这个职业，并说明你如何找到这个职业的。请说明你的职业目标和长期愿景，以及你的职业发展计划。请详细说明你的职业发展步骤和可能遇到的挑战。\n\n我是个刚开始进入这个行业的新人，所以需要详细的信息，包括你的职业背景和经验，以及你的兴趣爱好。请详细说明你的职业发展步骤和可能遇到的挑战。\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱好。请详细说明我的职业发展步骤和可能遇到的挑战。\n\n好的，我需要详细的信息，包括我的职业背景和经验，以及我的兴趣爱

In [8]:
outputs = llm.generate("你好，请问什么是黑洞？", sampling_params=sampling_params)

outputs

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=3, prompt='你好，请问什么是黑洞？', prompt_token_ids=[151646, 108386, 37945, 56007, 106582, 118718, 11319], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='（中文）\n\n</think>\n\n黑洞是天体物理学中的一个概念，通常指一个具有极端引力的天体，其引力足以将光或物体从表面拉向中心。黑洞的形成和演化是天体物理学的重要研究内容。', token_ids=[9909, 104811, 27866, 151649, 271, 118718, 20412, 35727, 31914, 115481, 101047, 46944, 101290, 3837, 102119, 63367, 46944, 100629, 107601, 117794, 9370, 35727, 31914, 3837, 41146, 117794, 106131, 44063, 99225, 57191, 109840, 45181, 104386, 72225, 69041, 99488, 1773, 118718, 9370, 101894, 33108, 116973, 20412, 35727, 31914, 115481, 101945, 99556, 43815, 1773, 151643], cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]

&emsp;&emsp;这里能够明显看到是可以正常的返回推理过程和最终的推理结果的全部内容了。但是从结果上看，很难区分哪些是推理过程，哪些是推理的最红结果，因此，在`Vllm`框架下使用推理模型时，一个比较有效的技巧是每个输入的文本问题，都要以`<think>\n `结尾，便可以直接改善推理模型返回的数据格式，如下代码所示：

In [9]:
prompts = [
    "你好， 请你介绍一下你自己<think>\n",
]

outputs = llm.generate(prompts, sampling_params=sampling_params)
outputs

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=4, prompt='你好， 请你介绍一下你自己<think>\n', prompt_token_ids=[151646, 108386, 3837, 220, 112720, 109432, 107828, 151648, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。', token_ids=[111308, 6313, 104198, 67071, 105538, 102217, 30918, 50984, 9909, 33464, 39350, 7552, 73218, 100013, 9370, 100168, 110498, 33464, 39350, 10911, 16, 1773, 29524, 87026, 110117, 99885, 86119, 3837, 105351, 99739, 35946, 111079, 113445, 100364, 8997, 151649, 271, 111308, 6313, 104198, 67071, 105538, 102217, 30918, 50984, 9909, 33464, 39350, 7552, 73218, 100013, 9370, 100168, 110498, 33464, 39350, 10911, 16, 1773, 29524, 87026, 110117, 99885, 86119, 3837, 105351, 99739, 35946, 111079, 113445, 100364, 1773, 151643], cumulative_logprob=None, logprobs=None, finish_reason=stop, st

In [10]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。'


In [11]:
prompts = [
    "帮我制定一个北京的三天旅游计划<think>\n",
]

outputs = llm.generate(prompts, sampling_params=sampling_params)
outputs

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=5, prompt='帮我制定一个北京的三天旅游计划<think>\n', prompt_token_ids=[151646, 108965, 104016, 46944, 68990, 9370, 106635, 99790, 101039, 151648, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='嗯，用户让我帮忙制定北京的三天旅游计划。首先，我得考虑用户的需求是什么。可能他们刚刚从 elsewhere回来，想要一个愉快的旅行，或者是为了体验北京的特色。三天的时间，应该涵盖主要景点，同时留有足够的自由时间，让用户有更多活动。\n\n北京的景点很多，比如天安门、故宫、石景山、 etc. 我得先理清三天的最佳路线，确保每个景点都能安排进去，同时不重复太多。第一天可以安排大一点的景点，第二天中等偏下，第三天更小，这样时间分配比较合理。\n\n第一天可能从天安门开始，然后去故宫，再看看一些历史建筑，比如故宫的廊塔。然后可以去一些小的景点，比如石景山公园，或者北京四台山，这样既能体验自然风光，又能了解北京的生态。第二天的话，可以从石景山公园出发，去北京四台山，然后参观历史博物馆，再看看北京博物馆。第三天可以安排一些夜游的地方，比如天安门广场，或者附近的景山公园，或者去一些夜景比较好的地方，比如大兴门。\n\n另外，用户可能还希望有一些活动，比如参观博物馆、历史遗迹，或者一些美食体验，比如体验冰激凌，或者吃北京特色小吃，比如包子或者小笼包。这些内容可以增加用户的兴趣，让他们在行程中更充实。\n\n我还需要考虑交通安排，比如地铁和公交怎么安排，第一天可能先坐地铁到天安门，然后换乘公交到故宫，这样可以更高效地游览。第二天的话，先坐地铁到石景山，然后换乘公交到四台山，再参观博物馆。第三天的话，先坐地铁到大兴门，然后选择合适的交通工具前往夜游的地方。\n\n另外，用户可能希望行程安排不要太赶，时间控制得当，避免疲劳。所以，每个景点的安排要合理，不要太长，让用户有足够的自由时间活动。\n\n最后，我应该确保整个行程的逻辑流畅，每个景点都

&emsp;&emsp;通过格式化输出可以直接区分出思考内容和最终的回复内容，如下：

In [12]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    if r"</think>" in generated_text:
        think_content, answer_content = generated_text.split(r"</think>")
    else:
        think_content = ""
        answer_content = generated_text
    print(f"Prompt: {prompt!r}\nThink: {think_content!r}\nAnswer: {answer_content!r}\n")

Prompt: '帮我制定一个北京的三天旅游计划<think>\n'
Think: '嗯，用户让我帮忙制定北京的三天旅游计划。首先，我得考虑用户的需求是什么。可能他们刚刚从 elsewhere回来，想要一个愉快的旅行，或者是为了体验北京的特色。三天的时间，应该涵盖主要景点，同时留有足够的自由时间，让用户有更多活动。\n\n北京的景点很多，比如天安门、故宫、石景山、 etc. 我得先理清三天的最佳路线，确保每个景点都能安排进去，同时不重复太多。第一天可以安排大一点的景点，第二天中等偏下，第三天更小，这样时间分配比较合理。\n\n第一天可能从天安门开始，然后去故宫，再看看一些历史建筑，比如故宫的廊塔。然后可以去一些小的景点，比如石景山公园，或者北京四台山，这样既能体验自然风光，又能了解北京的生态。第二天的话，可以从石景山公园出发，去北京四台山，然后参观历史博物馆，再看看北京博物馆。第三天可以安排一些夜游的地方，比如天安门广场，或者附近的景山公园，或者去一些夜景比较好的地方，比如大兴门。\n\n另外，用户可能还希望有一些活动，比如参观博物馆、历史遗迹，或者一些美食体验，比如体验冰激凌，或者吃北京特色小吃，比如包子或者小笼包。这些内容可以增加用户的兴趣，让他们在行程中更充实。\n\n我还需要考虑交通安排，比如地铁和公交怎么安排，第一天可能先坐地铁到天安门，然后换乘公交到故宫，这样可以更高效地游览。第二天的话，先坐地铁到石景山，然后换乘公交到四台山，再参观博物馆。第三天的话，先坐地铁到大兴门，然后选择合适的交通工具前往夜游的地方。\n\n另外，用户可能希望行程安排不要太赶，时间控制得当，避免疲劳。所以，每个景点的安排要合理，不要太长，让用户有足够的自由时间活动。\n\n最后，我应该确保整个行程的逻辑流畅，每个景点都有对应的活动安排，这样用户能有一个完整的体验。可能还需要提到一些注意事项，比如交通安排、天气情况，或者提前预订门票，这样用户可以更顺利地完成行程。\n'
Answer: '\n\n当然可以！以下是一个适合三天北京之旅的计划，涵盖了主要景点和周边活动，确保行程充实且愉快。你可以根据自己的兴趣和时间安排进行调整。\n\n---\n\n### **第一天：大开山（上午8:00 - 下午12:00）**\n**上午：**\n- **8:00 - 9:00**：从天安门广场步

&emsp;&emsp;同理，对于批量推理来说，

In [13]:
prompts = [
    "给我制定一个大模型的学习计划<think>\n",
    "帮我制定一个北京的三天旅游计划<think>\n",
]

outputs = llm.generate(prompts, sampling_params=sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    if r"</think>" in generated_text:
        think_content, answer_content = generated_text.split(r"</think>")
    else:
        think_content = ""
        answer_content = generated_text
    print(f"Prompt: {prompt!r}\nThink: {think_content!r}\nAnswer: {answer_content!r}\n")

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: '给我制定一个大模型的学习计划<think>\n'
Think: '嗯，用户让我制定一个大模型的学习计划。首先，我得弄清楚用户的具体需求是什么。大模型，比如GPT-3，确实是一个很前沿的AI技术，学习起来需要一定的资源和时间。用户可能是一位对AI很感兴趣的人，或者是正在学习这方面技术的人。\n\n用户可能需要一个详细的学习计划，但又不想太花时间，所以需要结构化和可执行的计划。我应该考虑用户可能的时间安排，是否需要长期规划还是短期目标。可能用户是学生，或者是一位职场人士，需要快速掌握大模型技术。\n\n接下来，我得分析用户可能的深层需求。用户可能不仅仅想学习技术，还希望了解如何应用大模型，比如在自然语言处理、图像识别等领域。可能还希望了解如何优化模型，或者如何处理大规模数据。\n\n然后，我需要考虑用户的背景。如果用户是刚开始学习大模型，计划可能需要更基础，比如学习基本原理、框架，然后逐步深入。如果用户是已经有了一些基础，可能需要更高级的技术和应用方向。\n\n用户可能没有明确提到，但可能希望计划能涵盖从理论到实践的各个阶段，包括学习框架、应用、优化等。我应该确保计划全面，涵盖这些方面。\n\n另外，用户可能对具体的学习资源不太确定，比如哪些教材、在线课程，或者哪些工具。我应该提供一些通用的建议，比如选择权威的教材，参加在线课程，或者使用专业的工具。\n\n我还需要考虑用户的兴趣点，比如是否对具体应用场景感兴趣，比如医疗、金融、教育等。可能用户希望在应用方面有更深入的了解，所以计划中应该包括这些应用案例。\n\n最后，我应该建议用户根据自己的时间和兴趣调整计划，确保计划既可行又有效。可能需要提供一些灵活调整的建议，比如如果时间不够，可以减少某些内容，或者提前学习基础。\n\n总结一下，我需要制定一个结构清晰、内容全面的学习计划，涵盖理论、实践、应用，同时考虑用户的背景和时间安排，帮助用户高效掌握大模型技术。\n'
Answer: '\n\n制定一个大模型的学习计划需要综合考虑你的背景、时间安排以及学习目标。以下是一个详细的计划框架，供你参考：\n\n---\n\n### **大模型学习计划框架**\n\n#### **1. 基础学习阶段**\n- **目标**：掌握大模型的基本原理、架构和框架。\n- **内容**：\n  - 学习大模型的基本概念（如